
## Synthetic Data Generation Testing

Now that we have successfully tested the preprocessing pipeline, we will extend this notebook to include:
- **Synthetic Data Generation**
- **Saving and Visualizing the Generated Data**
- **Comparing Synthetic and Original Data**


In [1]:
import sys
import os

# Get the absolute path of the project root (move up from notebooks/tests)
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  

# Add `src/` directory explicitly to Python path
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)


In [2]:
# Verify the path
print(sys.path)

['c:\\Users\\delea\\OneDrive\\Documents\\Desktop\\Master Thesis\\MasterThesisCode\\src', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312', '', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\delea\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\Pythonwin']


In [3]:
import pandas as pd
from preprocessing.data_loader import load_dataset
from preprocessing.missing_value_handler import handle_missing_values
from preprocessing.encoding import encode_categorical_features

# Define dataset path
original_dataset_path = "../../datasets/original/loan.csv"
separator = ","  # Adjust based on dataset format
target_column = "LoanAmount"  # Adjust based on dataset


In [4]:
# Load dataset
original_data, dataset_name = load_dataset(original_dataset_path, separator)
original_data.head()


📂 Loading dataset from: ../../datasets/original/loan.csv...

Processing dataset: loan
Original dataset size: 614 rows


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
# Handle missing values
cleaned_data = handle_missing_values(original_data, strategy="drop")
cleaned_data.head()


Dropped 134 rows due to missing values


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [6]:

# Encode categorical features using Binary Encoding
encoded_data = encode_categorical_features(cleaned_data, target_column)
encoded_data.head()


🔹 Identified Categorical Columns: ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']
Original Data Shape: (480, 13)
✅ Binary Encoding applied. New Features Added: ['Loan_ID_4', 'Property_Area_1', 'Loan_ID_1', 'Education_0', 'Gender_0', 'Loan_ID_7', 'Self_Employed_0', 'Loan_Status_0', 'Property_Area_0', 'Gender_1', 'Loan_ID_6', 'Loan_Status_1', 'Education_1', 'Loan_ID_2', 'Self_Employed_1', 'Dependents_1', 'Loan_ID_0', 'Loan_ID_8', 'Married_0', 'Dependents_2', 'Loan_ID_3', 'Loan_ID_5', 'Dependents_0', 'Married_1']
New Data Shape after Encoding: (480, 29)


,Loan_ID_0,Loan_ID_1,Loan_ID_2,Loan_ID_3,Loan_ID_4,Loan_ID_5,Loan_ID_6,Loan_ID_7,Loan_ID_8,Gender_0,...,Self_Employed_1,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_0,Property_Area_1,Loan_Status_0,Loan_Status_1
1,0,0,0,0,0,0,0,0,1,0,...,1,4583,1508.0,128.0,360.0,1.0,0,1,0,1
2,0,0,0,0,0,0,0,1,0,0,...,0,3000,0.0,66.0,360.0,1.0,1,0,1,0
3,0,0,0,0,0,0,0,1,1,0,...,1,2583,2358.0,120.0,360.0,1.0,1,0,1,0
4,0,0,0,0,0,0,1,0,0,0,...,1,6000,0.0,141.0,360.0,1.0,1,0,1,0
5,0,0,0,0,0,0,1,0,1,0,...,0,5417,4196.0,267.0,360.0,1.0,1,0,1,0



## Synthetic Data Generation Testing

Now that we have successfully tested the preprocessing pipeline, we will extend this notebook to include:
- **Synthetic Data Generation**
- **Saving and Visualizing the Generated Data**
- **Comparing Synthetic and Original Data**


In [7]:

# Import necessary libraries
import os
import pandas as pd
from synthetic_pipeline.data_synthesis import generate_synthetic_data, load_or_train_synthesizer

# Define parameters
TEST_SIZE = 0.2
DATASET_NAME = "loan"
SYNTHETIC_DATA_DIR = "datasets/synthetic"

# Generate synthetic data using the preprocessed dataset
synthetic_data, metadata = generate_synthetic_data(encoded_data, DATASET_NAME, TEST_SIZE)

# Display synthetic data preview
synthetic_data.head()



Processing dataset: loan
Original dataset size: 480 rows

 No existing synthesizer for loan found.
Training new synthesizer... (this may take a while)


c:\Users\delea\AppData\Local\Programs\Python\Python312\Lib\site-packages\sdv\single_table\base.py:97: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


 New synthesizer trained and saved successfully.

Using synthesizer 'loan_synthesizer' to generate synthetic data for dataset 'loan'
Synthetic data generated successfully: 384 rows created
Synthetic data saved to c:\Users\delea\OneDrive\Documents\Desktop\Master Thesis\MasterThesisCode\datasets\synthetic\loan_synthetic.csv


,Loan_ID_0,Loan_ID_1,Loan_ID_2,Loan_ID_3,Loan_ID_4,Loan_ID_5,Loan_ID_6,Loan_ID_7,Loan_ID_8,Gender_0,...,Self_Employed_1,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_0,Property_Area_1,Loan_Status_0,Loan_Status_1
0,0,0,0,0,1,1,1,1,1,0,...,1,150,2365.502161,24.0,360.0,1.0,1,0,1,0
1,0,1,0,1,1,0,0,0,0,0,...,1,2335,4199.919212,97.0,360.0,1.0,0,0,1,1
2,1,0,0,1,1,0,0,1,0,0,...,1,150,2703.545759,9.0,360.0,1.0,0,1,1,0
3,1,1,1,1,1,0,1,1,0,0,...,1,150,2205.280313,9.0,360.0,1.0,1,0,1,0
4,1,1,1,0,1,1,0,0,1,0,...,1,150,615.172351,9.0,360.0,1.0,1,0,1,0



## Saving Synthetic Data

The generated synthetic data will be saved into the `datasets/synthetic` directory. Let's verify that it is correctly stored.


In [ ]:

# Save synthetic data to CSV
synthetic_data_path = os.path.join(SYNTHETIC_DATA_DIR, f"{DATASET_NAME}_synthetic.csv")
synthetic_data.to_csv(synthetic_data_path, index=False)

# Check if the file exists
os.path.exists(synthetic_data_path)



## Comparing Synthetic vs Original Data

We will compare key statistics of the original and synthetic datasets to evaluate how well the synthetic data replicates the original distribution.


In [ ]:

# Compare basic statistics of original vs synthetic data
original_stats = cleaned_data.describe()
synthetic_stats = synthetic_data.describe()

# Display comparison
display(original_stats, synthetic_stats)
